In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Read and create data frame for Border Entries file
bd_ent = pd.read_csv('Border_Crossing_Entry_Data.csv')
bd_ent

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524)
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207)
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333)
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996)
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005)
5,Madawaska,Maine,109,US-Canada Border,03/01/2019 12:00:00 AM,Trucks,179,POINT (-68.3271 47.35446)
6,Pembina,North Dakota,3401,US-Canada Border,03/01/2019 12:00:00 AM,Bus Passengers,1054,POINT (-97.24333 48.96639)
7,Progreso,Texas,2309,US-Mexico Border,03/01/2019 12:00:00 AM,Truck Containers Empty,1808,POINT (-97.94889 26.061670000000003)
8,Portal,North Dakota,3403,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Empty,6685,POINT (-102.54917 48.99583)
9,Champlain-Rouses Point,New York,712,US-Canada Border,03/01/2019 12:00:00 AM,Trucks,24759,POINT (-73.44694 44.98639)


In [3]:
# Read and create data frame for Trade file
trade = pd.read_csv('COW_Trade_3.0/dyadic_trade_3.0.csv')
trade.head()

,ccode1,ccode2,year,importer1,importer2,flow1,flow2,source1,source2,bel_lux_alt_flow1,bel_lux_alt_flow2,china_alt_flow1,china_alt_flow2,version
0,2.0,20,1920,United States of America,Canada,611.86,735.48,1,1,-9.0,-9.0,-9.0,-9.0,2.01
1,2.0,20,1921,United States of America,Canada,335.44,442.99,1,1,-9.0,-9.0,-9.0,-9.0,2.01
2,2.0,20,1922,United States of America,Canada,364.02,502.84,1,1,-9.0,-9.0,-9.0,-9.0,2.01
3,2.0,20,1923,United States of America,Canada,416.00,598.14,1,1,-9.0,-9.0,-9.0,-9.0,2.01
4,2.0,20,1924,United States of America,Canada,399.14,496.32,1,1,-9.0,-9.0,-9.0,-9.0,2.01


In [4]:
# clean trade dataframe to only get data on us imports from canada
can_export = trade.loc[(trade['importer1'] == 'United States of America') & (trade['importer2'] == 'Canada')]
can_export.head()
#flow 1 is from importer 2 into importer 1

,ccode1,ccode2,year,importer1,importer2,flow1,flow2,source1,source2,bel_lux_alt_flow1,bel_lux_alt_flow2,china_alt_flow1,china_alt_flow2,version
0,2.0,20,1920,United States of America,Canada,611.86,735.48,1,1,-9.0,-9.0,-9.0,-9.0,2.01
1,2.0,20,1921,United States of America,Canada,335.44,442.99,1,1,-9.0,-9.0,-9.0,-9.0,2.01
2,2.0,20,1922,United States of America,Canada,364.02,502.84,1,1,-9.0,-9.0,-9.0,-9.0,2.01
3,2.0,20,1923,United States of America,Canada,416.00,598.14,1,1,-9.0,-9.0,-9.0,-9.0,2.01
4,2.0,20,1924,United States of America,Canada,399.14,496.32,1,1,-9.0,-9.0,-9.0,-9.0,2.01


In [5]:
# clean trade dataframe to only get data on us imoprts from mexico
mex_export = trade.loc[(trade['importer1'] == 'United States of America') & (trade['importer2'] == 'Mexico')]
mex_export.head()
#flow 1 is from importer 2 into importer 1

,ccode1,ccode2,year,importer1,importer2,flow1,flow2,source1,source2,bel_lux_alt_flow1,bel_lux_alt_flow2,china_alt_flow1,china_alt_flow2,version
790,2.0,70,1870,United States of America,Mexico,3.0,6.0,1,1,-9.0,-9.0,-9.0,-9.0,2.01
791,2.0,70,1871,United States of America,Mexico,3.0,8.0,1,1,-9.0,-9.0,-9.0,-9.0,2.01
792,2.0,70,1872,United States of America,Mexico,-9.0,-9.0,-9,-9,-9.0,-9.0,-9.0,-9.0,2.01
793,2.0,70,1873,United States of America,Mexico,4.0,6.0,1,1,-9.0,-9.0,-9.0,-9.0,2.01
794,2.0,70,1874,United States of America,Mexico,4.0,6.0,1,1,-9.0,-9.0,-9.0,-9.0,2.01


In [6]:
# append mexico export data frame to canada export data frame
complete_nafta_trade = can_export.append(mex_export)
complete_nafta_trade
#flow 1 is entering millions of dollars into US from 'importer 2'

,ccode1,ccode2,year,importer1,importer2,flow1,flow2,source1,source2,bel_lux_alt_flow1,bel_lux_alt_flow2,china_alt_flow1,china_alt_flow2,version
0,2.0,20,1920,United States of America,Canada,611.86,735.48,1,1,-9.0,-9.0,-9.0,-9.0,2.01
1,2.0,20,1921,United States of America,Canada,335.44,442.99,1,1,-9.0,-9.0,-9.0,-9.0,2.01
2,2.0,20,1922,United States of America,Canada,364.02,502.84,1,1,-9.0,-9.0,-9.0,-9.0,2.01
3,2.0,20,1923,United States of America,Canada,416.00,598.14,1,1,-9.0,-9.0,-9.0,-9.0,2.01
4,2.0,20,1924,United States of America,Canada,399.14,496.32,1,1,-9.0,-9.0,-9.0,-9.0,2.01
5,2.0,20,1925,United States of America,Canada,473.72,608.35,1,1,-9.0,-9.0,-9.0,-9.0,2.01
6,2.0,20,1926,United States of America,Canada,974.98,609.72,1,1,-9.0,-9.0,-9.0,-9.0,2.01
7,2.0,20,1927,United States of America,Canada,475.02,718.35,1,1,-9.0,-9.0,-9.0,-9.0,2.01
8,2.0,20,1928,United States of America,Canada,488.99,866.32,1,1,-9.0,-9.0,-9.0,-9.0,2.01
9,2.0,20,1929,United States of America,Canada,503.00,845.75,1,1,-9.0,-9.0,-9.0,-9.0,2.01


In [7]:
nafta_trade = complete_nafta_trade[['year', 'importer1', 'importer2', 'flow1']].copy()
nafta_trade

,year,importer1,importer2,flow1
0,1920,United States of America,Canada,611.86
1,1921,United States of America,Canada,335.44
2,1922,United States of America,Canada,364.02
3,1923,United States of America,Canada,416.00
4,1924,United States of America,Canada,399.14
5,1925,United States of America,Canada,473.72
6,1926,United States of America,Canada,974.98
7,1927,United States of America,Canada,475.02
8,1928,United States of America,Canada,488.99
9,1929,United States of America,Canada,503.00


In [8]:
nafta_trade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 929
Data columns (total 4 columns):
year         230 non-null int64
importer1    230 non-null object
importer2    230 non-null object
flow1        230 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.0+ KB


In [10]:
bd_ent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346733 entries, 0 to 346732
Data columns (total 8 columns):
Port Name    346733 non-null object
State        346733 non-null object
Port Code    346733 non-null int64
Border       346733 non-null object
Date         346733 non-null object
Measure      346733 non-null object
Value        346733 non-null int64
Location     346733 non-null object
dtypes: int64(2), object(6)
memory usage: 21.2+ MB


In [11]:
bd_ent['Year'] = pd.to_datetime(bd_ent['Date'])

In [12]:
bd_ent.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524),2019-03-01
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207),2019-03-01
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333),2019-03-01
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996),2019-03-01
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005),2019-03-01


In [15]:
bd_ent['Year'] = bd_ent['Year'].dt.year
bd_ent.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524),2019
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207),2019
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333),2019
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996),2019
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005),2019


In [34]:
gb_year_df = bd_ent.groupby(bd_ent['Year'])

In [19]:
gb_year_df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524),2019
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207),2019
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333),2019
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996),2019
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005),2019
2364,Van Buren,Maine,108,US-Canada Border,12/01/2018 12:00:00 AM,Trucks,1204,POINT (-67.94271 47.16207),2018
2365,Maida,North Dakota,3416,US-Canada Border,12/01/2018 12:00:00 AM,Trucks,170,POINT (-98.36953 48.98568),2018
2366,Douglas,Arizona,2601,US-Mexico Border,12/01/2018 12:00:00 AM,Buses,210,POINT (-109.54472 31.344439999999995),2018
2367,Presidio,Texas,2403,US-Mexico Border,12/01/2018 12:00:00 AM,Bus Passengers,238,POINT (-104.37167 29.56056),2018
2368,Anacortes,Washington,3010,US-Canada Border,12/01/2018 12:00:00 AM,Personal Vehicle Passengers,5350,POINT (-122.61738999999999 48.49988),2018


In [24]:
len(gb_year_df)

24

In [31]:
gb_year_df.tail()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year
2359,Fort Fairfield,Maine,107,US-Canada Border,01/01/2019 12:00:00 AM,Truck Containers Full,39,POINT (-67.82965 46.76891),2019
2360,Vanceboro,Maine,105,US-Canada Border,01/01/2019 12:00:00 AM,Train Passengers,54,POINT (-67.42955 45.55984),2019
2361,Roseau,Minnesota,3426,US-Canada Border,01/01/2019 12:00:00 AM,Personal Vehicles,2705,POINT (-95.81074 48.77518),2019
2362,El Paso,Texas,2402,US-Mexico Border,01/01/2019 12:00:00 AM,Pedestrians,582606,POINT (-106.48639 31.758610000000004),2019
2363,Sumas,Washington,3009,US-Canada Border,01/01/2019 12:00:00 AM,Buses,92,POINT (-122.26361 49.00028),2019
11893,Presidio,Texas,2403,US-Mexico Border,01/01/2018 12:00:00 AM,Truck Containers Full,476,POINT (-104.37167 29.56056),2018
11894,Frontier,Washington,3020,US-Canada Border,01/01/2018 12:00:00 AM,Trucks,1972,POINT (-117.78134000000001 48.910160000000005),2018
11895,Roseau,Minnesota,3426,US-Canada Border,01/01/2018 12:00:00 AM,Truck Containers Empty,110,POINT (-95.81074 48.77518),2018
11896,Houlton,Maine,106,US-Canada Border,01/01/2018 12:00:00 AM,Personal Vehicles,13330,POINT (-67.84083 46.12611),2018
11897,Portal,North Dakota,3403,US-Canada Border,01/01/2018 12:00:00 AM,Truck Containers Full,6620,POINT (-102.54917 48.99583),2018


In [32]:
gb_year_df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524),2019
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207),2019
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333),2019
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996),2019
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005),2019
2364,Van Buren,Maine,108,US-Canada Border,12/01/2018 12:00:00 AM,Trucks,1204,POINT (-67.94271 47.16207),2018
2365,Maida,North Dakota,3416,US-Canada Border,12/01/2018 12:00:00 AM,Trucks,170,POINT (-98.36953 48.98568),2018
2366,Douglas,Arizona,2601,US-Mexico Border,12/01/2018 12:00:00 AM,Buses,210,POINT (-109.54472 31.344439999999995),2018
2367,Presidio,Texas,2403,US-Mexico Border,12/01/2018 12:00:00 AM,Bus Passengers,238,POINT (-104.37167 29.56056),2018
2368,Anacortes,Washington,3010,US-Canada Border,12/01/2018 12:00:00 AM,Personal Vehicle Passengers,5350,POINT (-122.61738999999999 48.49988),2018


In [61]:
measures = bd_ent.groupby(['Year','Border','Measure'])['Value'].sum()

In [62]:
measures
#each item in the measure column here will become a column in the nafta_trade df, with the values shown below inserted
# into each row for the given 

Year  Border            Measure                    
1996  US-Canada Border  Bus Passengers                   3870081
                        Buses                             173279
                        Pedestrians                       607987
                        Personal Vehicle Passengers    101070734
                        Personal Vehicles               39531000
                        Rail Containers Empty             124007
                        Rail Containers Full              329983
                        Train Passengers                  213596
                        Trains                             31457
                        Truck Containers Empty            235054
                        Truck Containers Full            1420629
                        Trucks                           5431096
      US-Mexico Border  Bus Passengers                   1943697
                        Buses                             119510
                        Pedestrians   

In [63]:
measures = measures.reset_index()

In [64]:
measures.head()

,Year,Border,Measure,Value
0,1996,US-Canada Border,Bus Passengers,3870081
1,1996,US-Canada Border,Buses,173279
2,1996,US-Canada Border,Pedestrians,607987
3,1996,US-Canada Border,Personal Vehicle Passengers,101070734
4,1996,US-Canada Border,Personal Vehicles,39531000


In [65]:
len(measures)

576

In [66]:
measures

,Year,Border,Measure,Value
0,1996,US-Canada Border,Bus Passengers,3870081
1,1996,US-Canada Border,Buses,173279
2,1996,US-Canada Border,Pedestrians,607987
3,1996,US-Canada Border,Personal Vehicle Passengers,101070734
4,1996,US-Canada Border,Personal Vehicles,39531000
5,1996,US-Canada Border,Rail Containers Empty,124007
6,1996,US-Canada Border,Rail Containers Full,329983
7,1996,US-Canada Border,Train Passengers,213596
8,1996,US-Canada Border,Trains,31457
9,1996,US-Canada Border,Truck Containers Empty,235054


In [67]:
nafta_trade.head()

,year,importer1,importer2,flow1
0,1920,United States of America,Canada,611.86
1,1921,United States of America,Canada,335.44
2,1922,United States of America,Canada,364.02
3,1923,United States of America,Canada,416.00
4,1924,United States of America,Canada,399.14


In [68]:
measures

,Year,Border,Measure,Value
0,1996,US-Canada Border,Bus Passengers,3870081
1,1996,US-Canada Border,Buses,173279
2,1996,US-Canada Border,Pedestrians,607987
3,1996,US-Canada Border,Personal Vehicle Passengers,101070734
4,1996,US-Canada Border,Personal Vehicles,39531000
5,1996,US-Canada Border,Rail Containers Empty,124007
6,1996,US-Canada Border,Rail Containers Full,329983
7,1996,US-Canada Border,Train Passengers,213596
8,1996,US-Canada Border,Trains,31457
9,1996,US-Canada Border,Truck Containers Empty,235054


In [74]:
per_border_values = measures.groupby(['Border'])
per_border_values.head()

,Year,Border,Measure,Value
0,1996,US-Canada Border,Bus Passengers,3870081
1,1996,US-Canada Border,Buses,173279
2,1996,US-Canada Border,Pedestrians,607987
3,1996,US-Canada Border,Personal Vehicle Passengers,101070734
4,1996,US-Canada Border,Personal Vehicles,39531000
12,1996,US-Mexico Border,Bus Passengers,1943697
13,1996,US-Mexico Border,Buses,119510
14,1996,US-Mexico Border,Pedestrians,34109364
15,1996,US-Mexico Border,Personal Vehicle Passengers,171522486
16,1996,US-Mexico Border,Personal Vehicles,62429373


Measure,Bus Passengers,Buses,Pedestrians,Personal Vehicle Passengers,Personal Vehicles,Rail Containers Empty,Rail Containers Full,Train Passengers,Trains,Truck Containers Empty,Truck Containers Full,Trucks
Year,,,,,,,,,,,,
1996,5813778,292789,34717351,272593220,101960373,268134,472219,224881,38966,1599429,3124031,8685180
1997,6896919,389969,44461186,307001980,119003203,354288,620145,260610,40541,1920118,3708600,9516639
1998,7608484,436803,45060023,314295857,120451297,513721,1079074,258624,41116,2405012,6223216,10217477
1999,7725590,469496,48801064,330891505,126689655,611927,1376950,265341,38949,2768834,7641245,11176168
2000,8338859,460056,47677158,329841500,129202573,684988,1481674,287756,40555,2747769,7684947,11573707
2001,7823231,457697,52253906,284076951,123834970,764043,1597954,272547,41046,2936782,7942907,11081868
2002,8139017,470321,51362762,268007308,122388232,774094,1653204,270242,40579,2913657,8340836,11342566
2003,7526855,475676,49604705,255834018,118313556,806863,1668857,246897,41911,2626413,8273120,10973782
2004,7278897,424729,48916697,254206275,121794376,835832,1790382,236141,41111,2622197,8666091,11407570


In [60]:
nafta_trade

,year,importer1,importer2,flow1
0,1920,United States of America,Canada,611.86
1,1921,United States of America,Canada,335.44
2,1922,United States of America,Canada,364.02
3,1923,United States of America,Canada,416.00
4,1924,United States of America,Canada,399.14
5,1925,United States of America,Canada,473.72
6,1926,United States of America,Canada,974.98
7,1927,United States of America,Canada,475.02
8,1928,United States of America,Canada,488.99
9,1929,United States of America,Canada,503.00
